In [1]:
from load import *
from torch import Tensor, nn
import torch
from anim import *
from model_base import Model, Trainer
from modules import *
from derivative import NormalizedPoissonRMSE

ROOT = "./Datas/Week 8"

Q = 1.60217663e-19

In [2]:
sc = load_space_charge() * -Q
ep = load_elec_potential()
vg = load_vgs()
ma = load_materials()
co = load_contacts()
sx, sy = load_spacing()

In [3]:
def extract(ep, sc):
    xep = ep.reshape(-1, 129, 17)
    xsc = sc.reshape(-1, 129, 17)
            
    ep_region_2 = xep[:, 45:84,:11].reshape(-1, 429)
    ep_region_5 = xep[:, 45:84,11:].reshape(-1, 234)
    sc_region_2 = xsc[:, 45:84,:11].reshape(-1, 429)

    joined = torch.cat([ep_region_2, ep_region_5, sc_region_2], dim = 1)

    return joined

def reconstruct(x, xep, xsc):
    ep_region_2 = x[:, :429].reshape(-1, 39, 11)
    ep_region_5 = x[:, 429:663].reshape(-1, 39, 6)
    sc_region_2 = x[:, 663:].reshape(-1, 39, 11)

    xep = xep.clone()
    xep[:, 45:84,:11] = ep_region_2
    xep[:, 45:84,11:] = ep_region_5
    xep = xep.reshape(-1, 129, 17)

    xsc = xsc.clone()
    xsc[:, 45:84,:11] = sc_region_2
    xsc = xsc.reshape(-1, 129, 17)

    return xep, xsc

class PoissonFixModel(Model):
    def __init__(self, device = None):
        self.fc = Sequential(
            Linear(1 + 4386 * 3, 4386),
            LeakySigmoid()
        )

    def forward(self, X: Tensor) -> Tensor:
        # This inner model should accept 1 + 4386 * 3 variables and output 4386 things
        cached_state_ = torch.zeros(3, 4386).to(Device()).double()
        results = torch.zeros(X.shape[0], 4386).to(Device()).double()

        poi = NormalizedPoissonRMSE()

        # Predict for each variable
        for i in range(X.shape[0]):
            # Predict
            x_ = torch.concat([X[i], cached_state_.reshape(-1)]).reshape(1, -1)
            x_ = self.fc(x_) # Shape should be 1, 4386

            # Gradient descent
            xep = x_[:, :2193].reshape(-1, 129, 17).detach()
            xsc = x_[:, 2193:].reshape(-1, 129, 17).detach()
            x = extract(xep, xsc).clone().detach()
            x.requires_grad = True
            optim = torch.optim.Adam([x])
            for _ in range(20):
                optim.zero_grad()
                rep, rsc = reconstruct(x, xep, xsc)
                poi_loss = poi(rep, rsc)
                poi_loss.backward()
                optim.step()

            # Reconstruct x
            rep, rsc = reconstruct(x, xep, xsc)
            x_[:, :2193] = rep.reshape(-1, 2193)
            x_[:, 2193:] = rsc.reshape(-1, 2193)
            
            # Store the cached state
            cached_state_[1:] = cached_state_[:-1].clone()
            cached_state_[0] = x_

            # Store the result
            results[i] = x_
        return results

In [4]:
from model_base import fit

class PFMTrainer(Trainer):
    def __init__(self):
        self.model = PoissonFixModel().to(Device())
        self.mse = MSELoss()
        self.poi = NormalizedPoissonRMSE()

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        x = self.model(x)
        mse = self.mse(x, y)
        poi = self.poi(x[:, :2193].reshape(-1, 129, 17), x[:, 2193:].reshape(-1, 129, 17))
        self.add_loss("MSE", mse.item())
        self.add_loss("Poisson", poi.item())
        return mse + poi

In [5]:
idx = util.TRAINING_IDXS["First 30"]
model_ = PFMTrainer()
y = torch.cat([ep.reshape(-1, 2193), sc.reshape(-1, 2193)], dim = 1)
model = fit(model_, vg, y, idx, epochs=500)

On epoch 500, train loss = 15.8862016, test loss = 17.2000832: 100%|██████████| 500/500 [1:38:38<00:00, 11.84s/it]


In [10]:
results = model.model(vg.to(Device()))

In [16]:
ypred = results[:, :2193].detach().cpu().numpy().reshape(-1, 129, 17)
y = ep.detach().numpy()

anim = AnimationMaker()
anim.add_data(ep, "Original")
anim.add_data(ypred, "Predicted data", vmin = ep.min().item(), vmax = ep.max().item())
anim.add_data(log_diff(ep, ypred), "Log difference")
rmse = np.sqrt(np.mean((ypred - y) ** 2))
worst = np.max(np.abs(ypred - y))
rmse_last_10_frames = np.sqrt(np.mean((ypred[-10:] - y[-10:]) ** 2))
worst_last_10_frames = np.max(np.abs(ypred[-10:] - y[-10:]))

error_text = f"RMSE: {round(rmse, 5)}, (last 10 frames): {round(rmse_last_10_frames, 5)}, worst = {round(worst, 5)} (last 10 frames) = {round(worst_last_10_frames, 5)}"
anim.add_text([error_text for _ in range(101)])

frames = [f"Frame {frame} - {round(frame*0.0075, 4)}V" for frame in range(101)]
anim.add_text(frames)

anim.save("./a.gif")

C:\Users\Darin\AppData\Local\Temp\ipykernel_18632\2681282300.py:8: RuntimeWarning: overflow encountered in square
  rmse = np.sqrt(np.mean((ypred - y) ** 2))


In [17]:
model.model.save("./a.hlpt")

In [18]:
m = PoissonFixModel.load("./a.hlpt")

OutOfMemoryError: CUDA out of memory. Tried to allocate 442.00 MiB (GPU 0; 4.00 GiB total capacity; 3.06 GiB already allocated; 0 bytes free; 3.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF